<a href="https://colab.research.google.com/github/Martin-Roberto-lecuona/concurrente-M1/blob/main/TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Directorios

In [ ]:
!mkdir TP1
!mkdir TP1/Parte-1
!mkdir TP1/Parte-2
!mkdir TP1/Parte-3

# Codigo en C

##Parte 1

#### Crear el archivo

In [ ]:
%%writefile TP1/Parte-1/tp1_p1_c++.cpp
#include <iostream>
#include <unistd.h>
#include <stdlib.h>
#include <string.h>
#include <sys/wait.h>
#include <sys/types.h>
#include <map>
#include <vector>

#define SLEEP_TIME 20

using namespace std;

struct ProcesoInfo
{
    char proc;
    pid_t pid;
    pid_t ppid;
};

void imprimir_info(char proc)
{
    ProcesoInfo p = {proc, getpid(), getppid()};
    cout << "Proceso: " << p.proc << ", PID: " << p.pid << ", PPID: " << p.ppid << endl;
}

void imprimir_error(char proc)
{
    cerr << "Error al crear proceso " << proc << endl;
    exit(EXIT_FAILURE);
}

void crear_arbol(const std::pair<const char, std::vector<char>> &p, std::map<char, std::vector<char>> &procesoRelaciones)
{
    char padre = p.first;
    imprimir_info(padre);

    for (auto const &hijo : p.second)
    {
        if (procesoRelaciones.find(hijo) != procesoRelaciones.end())
        {
            // hijo que es padre
            pid_t pid_padre = fork();
            if (pid_padre < 0)
                imprimir_error(padre);
            else if (pid_padre == 0)
            {
                crear_arbol(*procesoRelaciones.find(hijo), procesoRelaciones);
                wait(NULL);
                sleep(SLEEP_TIME);
                exit(EXIT_SUCCESS);
            }
        } else
        {
            // hijo que es hoja
            pid_t pid_hoja = fork();
            if (pid_hoja < 0)
                imprimir_error(hijo);
            else if (pid_hoja == 0)
            {
                imprimir_info(hijo);
                sleep(SLEEP_TIME);
                exit(EXIT_SUCCESS);
            }
        }
    }
}

int main(int argc, char *argv[])
{
    if (argc == 2 && (strcmp(argv[1], "-h") == 0 || strcmp(argv[1], "--help") == 0))
    {
        cout << "Este proceso crea un árbol de procesos" << endl;
        return EXIT_SUCCESS;
    }

    char raiz = 'A';
    map<char, vector<char>> procesoRelaciones =
    {
        {'A', {'B', 'C', 'D'}},
        {'B', {'E', 'F'}},
        {'F', {'H', 'I'}},
        {'I', {'J'}},
        {'D', {'G'}}
    };

    crear_arbol(*procesoRelaciones.find(raiz), procesoRelaciones);

    while (wait(NULL) > 0);

    return EXIT_SUCCESS;
}



Writing TP1/Parte-1/tp1_p1_c++.cpp


Ver si se creo

In [ ]:
!ls -laR | grep tp1_p1_c++.cpp

-rw-r--r-- 1 root root 2166 Apr 12 21:46 tp1_p1_c++.cpp


#### Compilacion

In [ ]:
!g++ -o tp1P1.bin TP1/Parte-1/tp1_p1_c++.cpp

#### Ejecucion en background

Opcional: Help

In [ ]:
!./tp1P1.bin -h

Este proceso crea un árbol de procesos


In [ ]:
!nohup ./tp1P1.bin 1>salidaC 2>/dev/null &

#### Ver arbol de procesos

In [ ]:
!ps -ef | grep tp1P1.bin

root         452       1  0 21:46 ?        00:00:00 ./tp1P1.bin
root         453     452  0 21:46 ?        00:00:00 ./tp1P1.bin
root         454     452  0 21:46 ?        00:00:00 ./tp1P1.bin
root         455     452  0 21:46 ?        00:00:00 ./tp1P1.bin
root         456     453  0 21:46 ?        00:00:00 ./tp1P1.bin
root         457     453  0 21:46 ?        00:00:00 ./tp1P1.bin
root         458     455  0 21:46 ?        00:00:00 ./tp1P1.bin
root         459     457  0 21:46 ?        00:00:00 ./tp1P1.bin
root         460     457  0 21:46 ?        00:00:00 ./tp1P1.bin
root         461     460  0 21:46 ?        00:00:00 ./tp1P1.bin
root         478     225  0 21:46 ?        00:00:00 /bin/bash -c ps -ef | grep tp1P1.bin
root         480     478  0 21:46 ?        00:00:00 grep tp1P1.bin


Cambie el ID del proceso que solo diga ./tp1P1.bin

In [ ]:
!pstree 452

tp1P1.bin─┬─tp1P1.bin─┬─tp1P1.bin
          │           └─tp1P1.bin─┬─tp1P1.bin
          │                       └─tp1P1.bin───tp1P1.bin
          ├─tp1P1.bin
          └─tp1P1.bin───tp1P1.bin


**Salida del programa**

In [ ]:
!cat salidaC

Proceso: A, PID: 452, PPID: 1
Proceso: B, PID: 453, PPID: 452
Proceso: D, PID: 455, PPID: 452
Proceso: F, PID: 457, PPID: 453
Proceso: C, PID: 454, PPID: 452
Proceso: G, PID: 458, PPID: 455
Proceso: H, PID: 459, PPID: 457
Proceso: E, PID: 456, PPID: 453
Proceso: I, PID: 460, PPID: 457
Proceso: J, PID: 461, PPID: 460


**Matar al proceso de ser necesario**

In [ ]:
!kill 22929

# Codigo en Rust




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Parte 1

### Dependencias
Instalamos el compilador de rust, el administrador de proyectos cargo, y la libreria a utilizar

In [ ]:
!sudo apt-get install rustc


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libstd-rust-1.75 libstd-rust-dev
Suggested packages:
  cargo llvm-17 lld-17 clang-17
The following NEW packages will be installed:
  libstd-rust-1.75 libstd-rust-dev rustc
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 91.4 MB of archives.
After this operation, 369 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libstd-rust-1.75 amd64 1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04 [46.3 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libstd-rust-dev amd64 1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04 [41.6 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 rustc amd64 1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04 [3,404 kB]
Fetched 91.4 MB in 2s (58.4 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-lik

In [ ]:
!sudo apt-get install cargo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  cargo-doc
The following NEW packages will be installed:
  cargo
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 6,700 kB of archives.
After this operation, 23.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 cargo amd64 1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04 [6,700 kB]
Fetched 6,700 kB in 0s (35.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected pac

In [ ]:
!cargo --version && rustc --version

cargo 1.75.0
rustc 1.75.0 (82e1608df 2023-12-21) (built from a source tarball)


### Crear el archivo

creamos un nuevo proyecto en donde se va a ejecutar el codigo

In [ ]:
!cargo new /content/TP1/Parte-1/ej1_tp1_rust

     Created binary (application) `/content/TP1/Parte-1/ej1_tp1_rust` package


agregamos la libreria externa necesaria para el código

In [ ]:
!cd /content/TP1/Parte-1/ej1_tp1_rust && cargo add nix --features process

    Updating crates.io index
      Adding nix v0.28.0 to dependencies.
             Features:
             + process
             34 deactivated features
    Updating crates.io index


creamos el archivo

In [ ]:
%%writefile /content/TP1/Parte-1/ej1_tp1_rust/src/main.rs
use nix::sys::wait::{wait};
use nix::unistd::{fork, ForkResult};
use std::collections::HashMap;
use std::io::{Error, ErrorKind};
use std::process::id;
use std::{os::unix::process::parent_id, process::Command};

fn main()
{
    let process_tree: HashMap<char, Vec<char>> = HashMap::from([
        ('A', vec!['B', 'C', 'D']),
        ('B', vec!['E', 'F']),
        ('D', vec!['G']),
        ('F', vec!['H', 'I']),
        ('I', vec!['J']),
    ]);

    let seconds = 10;

    if let Err(e) = create_fork(&process_tree, &'A', &seconds)
    {
        eprintln!("Error: {}", e);
        std::process::exit(1);
    }
}

fn create_fork(
    process_tree: &HashMap<char, Vec<char>>,
    initial_node: &char,
    seconds: &u32,
) -> Result<(), Error>
{
    println!("Node {} id {} pid {}", initial_node, id(), parent_id());
    sleep_seconds(&1);

    match process_tree.get(&initial_node)
     {
        Some(children) =>
        {
            for &next_child in children
            {
                match unsafe
                {
                  fork()
                }
                {
                    Ok(ForkResult::Parent { .. }) => {}
                    Ok(ForkResult::Child) =>
                    {
                        if let Err(e) = create_fork(process_tree, &next_child, seconds)
                        {
                            eprintln!("Error in child node: {}", e);
                            std::process::exit(1)
                        }
                        std::process::exit(0)
                    }
                    Err(e) => return Err(Error::new(ErrorKind::Other, format!("Error in fork: {}", e))),
                }
            }
            for _ in 0..children.len()
            {
                let _ = wait();
            }
        }
        None => sleep_seconds(seconds),
    }

    println!("Ending node: {}", initial_node);
    Ok(())
}

fn sleep_seconds(seconds: &u32)
{
    Command::new("sleep")
        .arg(&seconds.to_string())
        .output()
        .expect("failure to execute process");
}

Overwriting /content/TP1/Parte-1/ej1_tp1_rust/src/main.rs


### Compilacion

In [ ]:
!cd /content/TP1/Parte-1/ej1_tp1_rust && cargo build

  Downloaded bitflags v2.5.0
  Downloaded cfg_aliases v0.1.1
  Downloaded cfg-if v1.0.0
  Downloaded nix v0.28.0
  Downloaded libc v0.2.153
   Compiling libc v0.2.153
   Compiling cfg_aliases v0.1.1
   Compiling nix v0.28.0
   Compiling cfg-if v1.0.0
   Compiling bitflags v2.5.0
   Compiling ej1_tp1_rust v0.1.0 (/content/TP1/Parte-1/ej1_tp1_rust)
    Finished dev [unoptimized + debuginfo] target(s) in 7.63s


### Ejecución en segundo plano

In [ ]:
!nohup /content/TP1/Parte-1/ej1_tp1_rust/target/debug/ej1_tp1_rust 1>/content/TP1/Parte-1/salidaRust 2>/dev/null &

### visualización de arbol de procesos

In [ ]:
!pstree -p | grep ej1_tp1_rust

docker-init(1)-+-ej1_tp1_rust(2799)-+-ej1_tp1_rust(2805)-+-ej1_tp1_rust(2816)---sleep(2819)
               |                    |                    `-ej1_tp1_rust(2817)---sleep(2821)
               |                    |-ej1_tp1_rust(2806)---sleep(2820)
               |                    `-ej1_tp1_rust(2808)---ej1_tp1_rust(2815)---sleep(2818)


visualizacion output

In [ ]:
!cat /content/TP1/Parte-1/salidaRust

Node A id 2799 pid 1
Node B id 2805 pid 2799
Node D id 2808 pid 2799
Node C id 2806 pid 2799
Node G id 2815 pid 2808
Node E id 2816 pid 2805
Node F id 2817 pid 2805
Node I id 2831 pid 2817
Node H id 2830 pid 2817
Node J id 2840 pid 2831
Ending node: C
Ending node: E
Ending node: G
Ending node: D
Ending node: H
Ending node: J
Ending node: I
Ending node: F
Ending node: B
Ending node: A


# Código en JAVA




## Parte 1

### Creación archivo

In [ ]:
%%writefile TP1/Parte-1/tp1_p1_java.java
import java.io.IOException;
import java.util.ArrayList;
import java.util.Map;
import java.util.Set;

class ProcessTree
{
  static Map<Character, Set<Character>> processMap = Map.of (
      'A', Set.of('B', 'C', 'D'),
      'B', Set.of('E', 'F'),
      'F', Set.of('H', 'I'),
      'I', Set.of('J'),
      'D', Set.of('G')
  );

  static ArrayList<Process> processList = new ArrayList<>();

  static final int SLEEP_TIME = 100000;
  // Milliseconds

  public static void showInfo ( char processName )
  {
    ProcessHandle me = ProcessHandle.current();
    long pid  = me.pid();
    long ppid = me.parent().get().pid();
    System.out.println( "Process " + processName + ", PID: " + pid
        + ", PPID: " + ppid );
  }

  public static void createChilds ( char processName ) throws IOException,
      InterruptedException
  {
    if ( processMap.containsKey(processName) )
    {
      for ( char son : processMap.get(processName) )
      {
        ProcessBuilder builder = new ProcessBuilder("java",
            "TP1/Parte-1/tp1_p1_java.java", String.valueOf(son));
        builder.inheritIO();
        Process process = builder.start();
        processList.add(process);
      }
    }
    else
    {
      Thread.sleep(SLEEP_TIME);
    }
  }

  public static void waitChilds ( ) throws InterruptedException
  {
    for ( Process p : processList )
    {
      p.waitFor();
    }
  }

  public static void main ( String[] args ) throws IOException,
      InterruptedException
  {
    char root = 'A';
    char process = ( args.length > 0 ) ? args[0].charAt(0) : root;
    showInfo(process);
    createChilds(process);
    waitChilds();
  }
}

Overwriting TP1/Parte-1/tp1_p1_java.java


### Ejecución en segundo plano

In [ ]:
!nohup java TP1/Parte-1/tp1_p1_java.java 1>salidaJava.txt 2>/dev/null &

### Lista de procesos en ejecución

In [ ]:
!ps -ef | grep tp1_p1_java.java

root        4771       1 47 22:01 ?        00:00:02 java TP1/Parte-1/tp1_p1_java.java
root        4793    4771 62 22:01 ?        00:00:01 java TP1/Parte-1/tp1_p1_java.java B
root        4795    4771 54 22:01 ?        00:00:01 java TP1/Parte-1/tp1_p1_java.java C
root        4798    4771 69 22:01 ?        00:00:02 java TP1/Parte-1/tp1_p1_java.java D
root        4865     225  0 22:01 ?        00:00:00 /bin/bash -c ps -ef | grep tp1_p1_java.java
root        4867    4865  0 22:01 ?        00:00:00 grep tp1_p1_java.java


### Árbol de procesos

In [ ]:
!pstree -pT 4771

java(4771)─┬─java(4793)─┬─java(4888)─┬─java(4948)───java(5004)
           │            │            └─java(4951)
           │            └─java(4891)
           ├─java(4795)
           └─java(4798)───java(4872)


### Salida del programa

In [ ]:
!cat salidaJava.txt

Process A, PID: 4771, PPID: 1
Process D, PID: 4798, PPID: 4771
Process B, PID: 4793, PPID: 4771
Process C, PID: 4795, PPID: 4771
Process G, PID: 4872, PPID: 4798
Process F, PID: 4888, PPID: 4793
Process E, PID: 4891, PPID: 4793
Process I, PID: 4948, PPID: 4888
Process H, PID: 4951, PPID: 4888
Process J, PID: 5004, PPID: 4948


# Código en GO


## Instalar paquete Goland

In [ ]:
!apt-get install -y golang-go

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  golang-1.18-go golang-1.18-src golang-src
Suggested packages:
  bzr | brz mercurial subversion
The following NEW packages will be installed:
  golang-1.18-go golang-1.18-src golang-go golang-src
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 82.3 MB of archives.
After this operation, 436 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-src all 1.18.1-1ubuntu1.1 [16.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-go amd64 1.18.1-1ubuntu1.1 [66.0 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 golang-src all 2:1.18~0ubuntu2 [4,438 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 golang-go amd64 2:1.18~0ubuntu2 [41.8 kB]
Fetched 82.3 MB in 3s (30.1 MB/s)
Selecting previously unselected pa

## Parte 1

### Crear Archivo

In [ ]:
%%writefile TP1/Parte-1/Parte1_go.go
package main

import (
	"fmt"
	"os"
	"os/exec"
	"sync"
	"time"
)

const RAIZ = 'A'
const ARGUMENTOS = 1
const CHILDLESS = 0
const SECONDS_WAIT = 10

type ProcessPrinter struct {
	mu sync.Mutex
}

var myDictionary = map[rune][]rune{
	'A': {'B', 'C', 'D'},
	'B': {'E', 'F'},
	'C': {},
	'D': {'G'},
	'E': {},
	'F': {'H', 'I'},
	'G': {},
	'H': {},
	'I': {'J'},
	'J': {},
}

var printer = ProcessPrinter{}

func main() {
	var wg sync.WaitGroup
	var process rune

	if len(os.Args) == ARGUMENTOS {
		process = rune(RAIZ)
	} else {
		process = rune(os.Args[1][0])
	}

	printer.showProcess(string(process))
	children, ok := myDictionary[process]

	if !ok {
		fmt.Println("Clave no encontrada en el diccionario")
		return
	}

	if len(children) > CHILDLESS {
		wg.Add(len(children))
		for _, letter := range children {
		   createChildProcess(letter, &wg)
		}
	}
	time.Sleep(SECONDS_WAIT * time.Second)
	wg.Wait()
}
func createChildProcess(name rune, wg *sync.WaitGroup) {
	defer wg.Done()

	cmd := exec.Command(os.Args[0], string(name))
	cmd.Stdout = os.Stdout
	cmd.Stderr = os.Stderr

	if err := cmd.Start(); err != nil {
		fmt.Println("Error al ejecutar el proceso hijo:", err)
	}
}
func (pp *ProcessPrinter) showProcess(process string) {
	pp.mu.Lock()
	defer pp.mu.Unlock()
	fmt.Println("Proceso", process, "pid:", os.Getpid(), "ppid:", os.Getppid())
}

Writing TP1/Parte-1/Parte1_go.go


### Compilacion



In [ ]:
!go run TP1/Parte-1/Parte1_go.go

Proceso A pid: 10183 ppid: 10144
Proceso B pid: 10187 ppid: 10183
Proceso D pid: 10188 ppid: 10183
Proceso C pid: 10198 ppid: 10183
Proceso F pid: 10199 ppid: 10187
Proceso E pid: 10200 ppid: 10187
Proceso G pid: 10208 ppid: 10188
Proceso I pid: 10212 ppid: 10199
Proceso H pid: 10213 ppid: 10199
Proceso J pid: 10226 ppid: 10212


### Ejecución en segundo plano

In [ ]:
!nohup go run TP1/Parte-1/Parte1_go.go 1>salidaGo.txt 2>/dev/null &

### Ver arbol de procesos

In [ ]:
!ps -axf | grep Parte1_go

   1217 ?        S      0:00  |       \_ /bin/bash -c ps -axf | grep Parte1_go
   1219 ?        S      0:00  |           \_ grep Parte1_go
   1139 ?        Sl     0:00 go run TP1/Parte-1/Parte1_go.go
   1169 ?        Sl     0:00  \_ /tmp/go-build1789597745/b001/exe/Parte1_go
   1174 ?        Sl     0:00      \_ /tmp/go-build1789597745/b001/exe/Parte1_go B
   1181 ?        Sl     0:00      |   \_ /tmp/go-build1789597745/b001/exe/Parte1_go E
   1188 ?        Sl     0:00      |   \_ /tmp/go-build1789597745/b001/exe/Parte1_go F
   1197 ?        Sl     0:00      |       \_ /tmp/go-build1789597745/b001/exe/Parte1_go H
   1201 ?        Sl     0:00      |       \_ /tmp/go-build1789597745/b001/exe/Parte1_go I
   1209 ?        Sl     0:00      |           \_ /tmp/go-build1789597745/b001/exe/Parte1_go J
   1175 ?        Sl     0:00      \_ /tmp/go-build1789597745/b001/exe/Parte1_go C
   1179 ?        Sl     0:00      \_ /tmp/go-build1789597745/b001/exe/Parte1_go D
   1192 ?        Sl     0:00    


### Arbol de procesos

In [ ]:
!pstree -pT 1169

Parte1_go(1169)─┬─Parte1_go(1174)─┬─Parte1_go(1181)
                │                 └─Parte1_go(1188)─┬─Parte1_go(1197)
                │                                   └─Parte1_go(1201)───Parte1_go(1209)
                ├─Parte1_go(1175)
                └─Parte1_go(1179)───Parte1_go(1192)


### Salida del programa

In [ ]:
!cat salidaGo.txt


Proceso A pid: 1169 ppid: 1139
Proceso B pid: 1174 ppid: 1169
Proceso C pid: 1175 ppid: 1169
Proceso D pid: 1179 ppid: 1169
Proceso E pid: 1181 ppid: 1174
Proceso F pid: 1188 ppid: 1174
Proceso G pid: 1192 ppid: 1179
Proceso H pid: 1197 ppid: 1188
Proceso I pid: 1201 ppid: 1188
Proceso J pid: 1209 ppid: 1201


## Parte 2

In [ ]:
%%writefile TP1/Parte-2/Parte2_go.go
package main

import (
	"fmt"
	"math/rand"
	"sync"
	"time"
)

const SIZE int = 9000
const THREADS int = 100

var matrixA [SIZE][SIZE]int
var matrixB [SIZE][SIZE]int
var matrixResSequ [SIZE][SIZE]int
var matrixResConc [SIZE][SIZE]int

func printMatrix(matrix *[SIZE][SIZE]int) {
	for i := 0; i < SIZE; i++ {
		for j := 0; j < SIZE; j++ {
			fmt.Print("|")
			if matrix[i][j] < 10 {
				fmt.Print(" ")
			}
			if matrix[i][j] < 100 {
				fmt.Print(" ")
			}
			fmt.Print(matrix[i][j])
			fmt.Print("|")
		}
		fmt.Println()
	}
}

func initMatrix(m *[SIZE][SIZE]int) {
	for i := 0; i < SIZE; i++ {
		for j := 0; j < SIZE; j++ {
			m[i][j] = rand.Intn(100)
		}
	}
}

func sumSeqMatrix() {
	for i := 0; i < SIZE; i++ {
		for j := 0; j < SIZE; j++ {
			matrixResSequ[i][j] = matrixA[i][j] + matrixB[i][j]
		}
	}
}
func sumConcMatrix() {
	var wg sync.WaitGroup
	wg.Add(THREADS)

	blockSize := SIZE / THREADS

	for i := 0; i < THREADS; i++ {
		go sumRows(i*blockSize, (i+1)*blockSize, &wg)
	}

	wg.Wait()
}

func sumRows(startRow int, endRow int, wg *sync.WaitGroup) {
	defer wg.Done()
	for r := startRow; r < endRow; r++ {
		for c := 0; c < SIZE; c++ {
			matrixResConc[r][c] = matrixA[r][c] + matrixB[r][c]
		}
	}
}

func compareRes() bool {
	for i := 0; i < SIZE; i++ {
		for j := 0; j < SIZE; j++ {
			if matrixResConc[i][j] != matrixResSequ[i][j] {
				fmt.Println(matrixResConc[i][j], "!=", matrixResSequ[i][j], "i", i, "j", j)
				return false
			}
		}
	}
	return true
}
func printAllMatrix() {
	fmt.Println("Matrix A:")
	printMatrix(&matrixA)
	fmt.Println("\nMatrix B:")
	printMatrix(&matrixB)

	fmt.Println("\nRes Seq:")
	printMatrix(&matrixResSequ)

	fmt.Println("\nRes Conc:")
	printMatrix(&matrixResConc)
}

func main() {
	fmt.Println("matrix size: ", SIZE)
	fmt.Println("Threads: ", THREADS)
	initMatrix(&matrixA)
	initMatrix(&matrixB)

	startSeq := time.Now()
	sumSeqMatrix()
	endSeq := time.Since(startSeq).Milliseconds()

	startConc := time.Now()
	sumConcMatrix()
	endConc := time.Since(startConc).Milliseconds()

	// printAllMatrix()

	fmt.Println("\nSECUENCIAL == CONCURRENTE:", compareRes())

	fmt.Println("SECUENCIAL: ", endSeq)
	fmt.Println("CONCURRENTE: ", endConc)
}


Overwriting TP1/Parte-2/Parte2_go.go


### Compilacion

In [ ]:
!go run TP1/Parte-2/Parte2_go.go

matrix size:  9000
Threads:  100

SECUENCIAL == CONCURRENTE: true
SECUENCIAL:  546
CONCURRENTE:  461


# Codigo en Python

## Parte 1

In [ ]:
%%writefile TP1/Parte-1/tp1_p1_python.py
import os
import time

TIME_SLEEP=20000


def imprimirProceso(proceso):
    pid = os.getpid()
    ppid = os.getppid()
    print(f"Proceso {proceso}: PID = {pid}, PID del Padre = {ppid}")

def crearProcesos(procesos, padre):
    hijos = procesos.get(padre, [])
    imprimirProceso(padre)
    for hijo in hijos:
        pid = os.fork()
        if pid < 0:
            print("Error pid del proceso")
        if pid == 0:  # es hijo
            crearProcesos(procesos, hijo)
            time.sleep(TIME_SLEEP)
            os._exit(0)
    for hijo in hijos:
        os.wait()


print("Comienzo a crear los procesos...\n")

procesos = {
    "A": ["B", "C", "D"],
    "B": ["E", "F"],
    "D": ["G"],
    "F": ["H", "I"],
    "I": ["J"]
}

crearProcesos(procesos, list(procesos.keys())[0])

print("\nCreación de procesos finalizada")

Writing TP1/Parte-1/tp1_p1_python.py


### Ver si se creo

In [ ]:
!ls -laR | grep tp1_p1_python.py

-rw-r--r-- 1 root root  828 Apr 11 18:29 tp1_p1_python.py


### Ejecución en segundo plano




In [ ]:
!nohup python3 TP1/Parte-1/tp1_p1_python.py &

nohup: appending output to 'nohup.out'


### Visualización de los procesos que están corriendo

In [ ]:
!ps -aufx

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root          22  0.0  0.0   5808  1000 ?        Ss   21:45   0:00 tail -n +0 -F /root/.config/Googl
root           1  0.0  0.0   1076     8 ?        Ss   21:45   0:00 /sbin/docker-init -- /datalab/run
root           6  0.5  0.3 886052 47516 ?        Rl   21:45   0:01 /tools/node/bin/node /datalab/web
root          16  0.0  0.0   7376  3428 ?        S    21:45   0:00  \_ /bin/bash -e /usr/local/colab
root        1057  0.0  0.0   5776  1000 ?        S    21:49   0:00  |   \_ sleep 1
root          61  3.3  0.0      0     0 ?        Z    21:45   0:07  \_ [python3] <defunct>
root          62  0.3  0.3  66516 51504 ?        S    21:45   0:00  \_ python3 /usr/local/bin/colab-
root          84  1.8  0.8 358404 112592 ?       Rl   21:45   0:03  \_ /usr/bin/python3 /usr/local/b
root         225  1.5  0.7 1109960 96668 ?       Ssl  21:45   0:02  |   \_ /usr/bin/python3 -m colab
root        1058  0.0  0.0  10076  1596 ?   

In [ ]:
!ps aux

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0   1076     8 ?        Ss   21:45   0:00 /sbin/docker-init -- /datalab/run
root           6  0.5  0.3 886052 47308 ?        Rl   21:45   0:01 /tools/node/bin/node /datalab/web
root          16  0.0  0.0   7376  3428 ?        S    21:45   0:00 /bin/bash -e /usr/local/colab/bin
root          18  0.0  0.0   7376  1864 ?        S    21:45   0:00 /bin/bash -e /datalab/run.sh
root          20  0.0  0.0 1235948 12892 ?       Sl   21:45   0:00 /usr/colab/bin/kernel_manager_pro
root          22  0.0  0.0   5808  1000 ?        Ss   21:45   0:00 tail -n +0 -F /root/.config/Googl
root          61  3.2  0.0      0     0 ?        Z    21:45   0:07 [python3] <defunct>
root          62  0.3  0.3  66516 51504 ?        S    21:45   0:00 python3 /usr/local/bin/colab-file
root          84  1.7  0.8 358404 112592 ?       Rl   21:45   0:03 /usr/bin/python3 /usr/local/bin/j
root          85  0.0  0.0 1227460 

##Visualización de procesos

In [ ]:
!ps -aux | grep TP1/Parte-1/tp1_p1_python.py

root        1126  0.0  0.0   6624  2208 ?        S    21:49   0:00 grep TP1/Parte-1/tp1_p1_python.py


### Visualización árbol de procesos

In [ ]:
!pstree -p 1000

python3(1000)─┬─python3(1001)─┬─python3(1004)
              │               └─python3(1005)─┬─python3(1007)
              │                               └─python3(1008)───python3(1009)
              ├─python3(1002)
              └─python3(1003)───python3(1006)


#Conclusiones

Luego de codificar la solución propuesta en distintos lenguajes pudimos notar diferencias entre ellos y además surgieron inconvenientes.

Por ejemplo, al codificar la solución en Python tuvimos conflicto con la identación, eso nos llevo a que el código no se ejecute correctamente y a obtener resultados inesperados. Por otra parte, en principio se desarrollo el código utilizando la biblioteca de Python llamada multiprocessing, que proporciona una interfaz de más alto nivel en comparación con la utilización de os.fork que es una interfaz de más bajo nivel. Respecto a esto una dificultad que encontramos es que fork no se puede ejecutar en Windows, mientras que multiprocessing si se puede. En este caso, preferimos utilizar os.fork ya que en el ejemplo dado en clase se utiliza os.fork, pero según lo que pudimos investigar solo funciona para Mac OS X, Unix y Linux, lo cual podría ser una desventaja ya que no asegura la portabilidad.

En el caso de Java, es un lenguaje de programación que tiene una sintaxis más compleja en comparación con Python, además de requerir más código para lograr el mismo resultado. Además, la ausencia de fork() puede presentar algunas limitaciones en la flexibilidad y eficiencia en comparación con C++, especialmente en escenarios donde la clonación exacta de un proceso es necesaria (cabe aclarar que aunque el espacio de memoria del proceso hijo es idéntico inicialmente al del proceso padre, cada uno tendrá su propia área de memoria y no se compartirá). Es por ello que Java ofrece alternativas como la clase ProcessBuilder, la cual permite configurar y ejecutar procesos externos, lo que facilita la interacción con programas externos y la comunicación a través de la entrada y salida estándar. Por otra parte, Java puede consumir más recursos en comparación con lenguajes de bajo nivel debido a la sobrecarga asociada con la JVM y la gestión de hilos, afectando de esta manera el rendimiento.

Durante el desarrollo en Rust, nos enfrentamos a varias dificultades, siendo una de las primeras la incorporación de la biblioteca externa "Nix" al programa. Esta biblioteca proporciona una alternativa de nivel superior para realizar operaciones con. La solución a este problema fue aprovechar del gestor de paquetes oficial de Rust, 'Cargo'. Al crear un proyecto en Cargo y agregar el código correspondiente, la incorporación de la biblioteca externa resultó relativamente sencilla. Por lo tanto, podemos afirmar que Cargo se convierte en una herramienta esencial para el desarrollo en este lenguaje.

Otro aspecto importante que debimos tener en cuenta fue la sintaxis particular del lenguaje. Aunque Rust está diseñado para que el código sea legible y seguro, al obligar al programador a ser más explícito, puede presentar una curva de aprendizaje más pronunciada para aquellos que no están familiarizados con él. Rust toma inspiración de la sintaxis de varios otros lenguajes de programación, como JavaScript, Python, C++ y Java.

Una de las particularidades que nos encontramos al desarrollar el código en GO es que no existe una sentencia similar a **fork()** , como las que conocimos en C y C++ en anteriores cursadas.

Para poder realizar programación concurrente en Go, utilizamos una característica sencilla propia del lenguaje que son la ejecucion de comandos. Asi es posible ejecutar varias tareas en forma concurrente sin preocuparse de la sobrecarga de recursos.
Con el *Cmd.Start()* es el método que inicia el comando especificado y no espera a que se complete.
La forma de utilizar los comandos es:
*cmd := exec.Command()*
